In [14]:
from crewai import Agent, Task, Crew
from IPython.display import Markdown
from crewai.llm import LLM


In [3]:
# Configura o LLM local via Ollama (model: gemma3:4b)
llm = LLM(model="ollama/gemma3:4b", base_url="http://localhost:11434")


In [ ]:
# Grupo De Agents Para Escrever Artigos
# 1. Planner - Planejador de Artigos
# 2. Writer - Escritor de Artigos
# 3. Editor - Editor de Artigos

In [5]:
planner = Agent(
    role="Planejador de Conteúdo",
    goal="Planejar conteúdo envolvente e factual sobre o {topic}",
    backstory="""
        Você está trabalhando no planejamento de um artigo de blog sobre o tópico: {topic}.
        Você coleta informações que ajudam o público a aprender algo e tomar decisões informadas.
        Seu trabalho é a base para o Escritor de Conteúdo escrever o artigo sobre esse tópico.
    """,
    allow_delegation=False,
    verbose=True,
    llm=llm,
)


In [6]:
writer = Agent(
    role="Escritor de Conteúdo",
    goal="Escrever um artigo opinativo informativo e baseado em fatos sobre o {topic}",
    backstory="""
        Você está escrevendo uma opinião sobre o tópico: {topic}.
        Você baseia sua opinião no trabalho do Planejador de Conteúdo, que fornece um roteiro e contexto relevante.
        Você segue os principais objetivos e a direção do roteiro, conforme fornecido pelo Planejador.
        Você também oferece insights objetivos e imparciais, apoiados por informações fornecidas pelo Planejador.
    """,
    allow_delegation=False,
    verbose=True,
    llm=llm,
)


In [7]:
editor = Agent(
    role="Editor",
    goal="Editar o artigo para alinhar com o estilo de escrita da organização.",
    backstory="""
        Você é um editor que recebe um artigo do Escritor de Conteúdo.
        Seu objetivo é revisar o artigo para garantir que segue as melhores práticas jornalísticas.
    """,
    allow_delegation=False,
    verbose=True,
    llm=llm,
)


In [9]:
plan = Task(
    description=(
        "1. Priorize as últimas tendências, principais players e notícias relevantes sobre {topic}."
        "2. Identifique o público-alvo, considerando seus interesses e dores."
        "3. Desenvolva um roteiro detalhado incluindo introdução, pontos-chave e chamada para ação."
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um plano de conteúdo abrangente com roteiro, análise de público, palavras-chave de SEO e recursos.",
    agent=planner,
)


In [11]:
write = Task(
    description=(
        "1. Use o plano de conteúdo para elaborar um artigo de blog atraente sobre {topic}."
        "2. Incorpore as palavras-chave de SEO de forma natural."
        "3. Nomeie seções/subtítulos de maneira envolvente e clara."
        "4. Garanta uma estrutura com introdução envolvente, corpo com insights e conclusão resumida."
    ),
    expected_output="Um artigo bem estruturado em Markdown, pronto para publicação; cada seção deve ter 2–3 parágrafos.",
    agent=writer,
)


In [12]:
edit = Task(
    description=(
        "Revisar o artigo de blog para correções gramaticais e alinhamento com a voz da marca."
    ),
    expected_output="Um artigo bem escrito em Markdown, pronto para publicação.",
    agent=editor,
)


In [13]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True,
)

In [ ]:
result = crew.kickoff(inputs={"topic": "O Futuro da IA na Saúde"})


In [ ]:
Markdown(result)